# Simple Twitter Analysis
### Includes: lemmatization, text preprocessing, twitter api request, sentiment analysis and basic dataframe operations
One of my first pieces of code. Twitter API credentials had expired by the time I uploaded it so there are no outputs.

In [ ]:
# Download any missing libraries (with pip)
import sys
!{sys.executable} -m pip install tweepy

In [ ]:
# Imports
import numpy as np
import re
import nltk
import textblob
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import tweepy 
import pandas as pd 

In [ ]:
# Your credentials go here. 
ckey="******"
csecret="******"
atoken="******"
asecret="******"

In [ ]:
auth = tweepy.OAuthHandler(ckey, csecret)
auth.set_access_token(atoken, asecret)
api = tweepy.API(auth)

In [ ]:
TRACKING_KEYWORDS = ["Cybertruck", "cybertruck", "CYBERTRUCK"] #<----- YOUR KEYWORDS GO HERE, in this example is searched for cybertruck related tweets
TWEETS_TO_CAPTURE = 2500 #MAX IS AROUND 3000 BUT SUBJECT TO TWITTER POLICY CHANGES. RETWEETS INCLUDED.


In [ ]:
data_raw = ["Initializer"]
location = [] #I HAVE CHOSEN TO ALSO GET THE LOCATION OF TWEETS, YOU CAN CHOOSE TO GET EXTRA INFO LIKE LANGUAGE, UPLOAD DEVICE ETC.
for status in tweepy.Cursor(api.search,q=TRACKING_KEYWORDS, tweet_mode="extended", lang="en").items(TWEETS_TO_CAPTURE):
    data_raw.append(status.full_text) # .full_text affects data type pulled, status alone grabs everything
    location.append(status.user.location)
del data_raw[0]
print ("Tweets downloaded", len(data_raw))

In [ ]:
#CREATES A DATAFRAME AND STORES ACQUIRED DATA THERE
df = pd.DataFrame(data=data_raw, index=None, columns=["text"], dtype=str, copy=False)
print(df)

In [ ]:
# DOWNLOADS ENGLISH STOPWORDS
stop_words=set(stopwords.words("english"))

In [ ]:
# PREPROCESSING
assistant = ""
for i in df.index.values:
    # LOWERING
    assistant = df.loc[i, "text"].lower()
    df.loc[i, "text"] = assistant.lower()
    
    # URL REMOVAL
    assistant = df.loc[i, "text"]
    assistant  = re.sub(r"http\S+", '', assistant)
    df.loc[i, "text"] = assistant
    
    # USER REMOVAL
    assistant = df.loc[i, "text"]
    assistant  = re.sub(r"(?:\@|https?\://)\S+", '', assistant)
    df.loc[i, "text"] = assistant
    
    # NUMBER & HTML REMOVAL
    assistant = df.loc[i, "text"]
    assistant  = re.sub('[0-9]+', '', assistant)
    assistant  = re.sub('&amp', '', assistant)
    df.loc[i, "text"] = assistant
    
    # EMOJI REMOVAL
    assistant = df.loc[i, "text"]
    assistant  = re.sub(r'[^\x00-\x7F]+', '', assistant)
    df.loc[i, "text"] = assistant
    
    # PUNCTUATION REMOVAL
    assistant = df.loc[i, "text"]
    assistant  = re.sub(r'[,.;:/\\#]', '', assistant)
    df.loc[i, "text"] = assistant
    
    # TOKENIZATION
    assistant = df.loc[i, "text"]
    assistant  = nltk.word_tokenize(assistant)
    df.loc[i, "text"] = assistant
   
    
    # REMOVE STOPWORDS & OTHER WORDS
    other = ["'s"]
    stop_words = list(stop_words)
    stop_words.extend(other)
    assistant = []
    for word in df.loc[i, "text"]:
        if word not in stop_words and word != "rt":
            assistant.append(word)
        df.loc[i, "text"] = assistant

        
    # LEMMATIZATION
    lemmatizer = WordNetLemmatizer()
    assistant = []
    for word in df.loc[i, "text"]:
        word = lemmatizer.lemmatize(word)
        assistant.append(word)
    df.loc[i, "text"] = assistant
    
    
    # REMAKING TWEET INTO STRING
    assistant = df.loc[i, "text"]
    assistant = " ".join(assistant)
    df.loc[i, "text"] = assistant

In [ ]:
# FINDS POLARITY OF EACH TWEET
assistant = ""
for i in df.index.values:
    blob = textblob.TextBlob(df.loc[i, "text"])
    df.loc[i, "sentiment"] = blob.sentiment.polarity

In [ ]:
# SORTING DATAFRAME BY SENTIMENT
df = df.sort_values(by='sentiment', ascending=True)

In [ ]:
# OVERALL SENTIMENT
assistant = 0
amount_of_tweets = len(df.index.values)
for i in df.index.values:
    assistant = df.loc[i, "sentiment"] + assistant
    
overal_sentiment = assistant / amount_of_tweets
print("THE OVERALL SENTIMENT IS",overal_sentiment )